# Monte-Carlo Off-Police

### Instalação de pacotes

In [2]:
from IPython.display import clear_output
!apt-get install ffmpeg freeglut3-dev xvfb  
!pip install gym[all]==00.25.1
!pip install gym[atari,accept-rom-license]==00.25.1
!pip install pyglet
!pip install stable-baselines3[extra]
!pip install optuna
clear_output()

In [3]:
!mkdir log_project

### Imports


In [4]:
import gym
import numpy as np
import tensorboard
import matplotlib.pyplot as plt
import time

%load_ext tensorboard

import gym
import numpy as np


### Para salvar vídeo

In [5]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

A gravação é feita com o wrapper [VecVideoRecorder](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder).

In [7]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id)])
  # Start the video at step=0 and record the given number of steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [8]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

# Plot Result

In [9]:
def test_greedy_Q_policy(env, Q, num_episodes=100, render=False, render_wait=0.01):
    """
    Avalia a política gulosa (greedy) definida implicitamente por uma Q-table.
    Ou seja, executa, em todo estado s, a ação "a = argmax Q(s,a)".
    - env: o ambiente
    - Q: a Q-table (tabela Q) que será usada
    - num_episodes: quantidade de episódios a serem executados
    - render: defina como True se deseja chamar `env.render()` a cada passo
    - render_wait: intervalo de tempo entre as chamadas a `env.render()`
    
    Retorna:
    - um par contendo o valor escalar do retorno médio por episódio e 
       a lista de retornos de todos os episódios
    """
    episode_returns = []
    total_steps = 0
    for i in range(num_episodes):
        print(f"Episode {i+1}")
        obs = env.reset()
        if render:
            env.render()
            time.sleep(render_wait)
        done = False
        episode_returns.append(0.0)
        while not done:
            action = np.argmax(Q[obs])
            obs, reward, done, _ = env.step(action)
            if render:
                env.render()
                time.sleep(render_wait)
            total_steps += 1
            episode_returns[-1] += reward
        print("- retorno:", episode_returns[-1])
    mean_return = round(np.mean(episode_returns), 1)
    print("Retorno médio (por episódio):", mean_return, end="")
    print(", episódios:", len(episode_returns), end="")
    print(", total de passos:", total_steps)
    return mean_return, episode_returns

In [10]:
def smooth(data, window):
  data = np.array(data)
  n = len(data)
  y = np.zeros(n)
  for i in range(n):
    start = max(0, i-window+1)
    y[i] = data[start:(i+1)].mean()
  return y

def plot_result(returns, ymax_suggested=None, window=100, filename=None):
    '''Exibe um gráfico "retornos x recompensas", fazendo a média a cada 100 retornos, para suavizar.     
    Se o parâmetro filename for fornecido, salva o gráfico em arquivo ao invés de exibir.
    
    Parâmetros:
    - returns: lista de retornos a cada episódio
    - ymax_suggested (opcional): valor máximo de retorno (eixo y), se tiver um valor máximo conhecido previamente
    - filename: indique um nome de arquivo, se quiser salvar a imagem do gráfico; senão, o gráfico será apenas exibido
    '''
    plt.figure(figsize=(14,8))
    smoothed_returns = smooth(returns, window)
    xvalues = np.arange(1, len(returns)+1)
    plt.plot(xvalues, smoothed_returns)
    plt.xlabel('Episódios')
    plt.ylabel('Retorno')
    if ymax_suggested is not None:
        ymax = np.max([ymax_suggested, np.max(smoothed_returns)])
        plt.ylim(top=ymax)
    plt.title(f"Retorno médio a cada {window} episódios")
    if filename is None:
        plt.show()
    else:
        plt.savefig(filename)
        print("Arquivo salvo:", filename)
    plt.close()


## Código

In [53]:
from numpy.random.mtrand import gamma

import optuna

#from wrappers import DiscreteObservationWrapper


ENV = gym.make("Taxi-v3")


# Esta função faz um treinamento com o Expected-SARSA, usando parâmetros sugeridos pelo Optuna.
# Retorna a média dos retornos dos últimos 100 episódios.
def train_values(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    #bins1 = trial.suggest_int('bins1', 5, 100)
    #bins2 = trial.suggest_int('bins2', 5, 100)
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}")

    # roda o algoritmo e recebe os retornos não-descontados
    #env_wrapper = DiscreteObservationWrapper(ENV, [bins1,bins2])
    (returns, _) = run_montecarloOffP(env, 20000, gamma, eps, render=False)
    return sum(returns[-100:])/100 

In [ ]:
study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_offpolice', 
                            load_if_exists=True)
study.optimize(train_values, n_trials=20) 

In [60]:
ENV_NAME = "Taxi-v3"  
#ENV_NAME = "MountainCarContinuous-v0"  
#ENV_NAME = "LunarLander-v2"  
env = gym.make(ENV_NAME)

### Off-policy

In [51]:
def choose_action(Q, state):
    return np.argmax(Q[state])

def choose_actionB(num_actions):
    return np.random.randint(0, num_actions)

# Algoritmo Monte-Carlo de Controle, variante "toda-visita".
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_montecarloOffP(env, episodes, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)
    
    num_actions = env.action_space.n
    
    # inicializa a tabela Q toda com zero,
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))
    C = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-discontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        ep_trajectory = []
        
        state = env.reset()
    
        # PARTE 1: executa um episódio completo
        while done != True:   
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios 
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()
                
            # escolhe a próxima ação -- usa epsilon-greedy
            action = choose_action(Q, state)
        
            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)
            
            # adiciona a tripla que representa este passo
            ep_trajectory.append( (state, action, reward) )
            
            sum_rewards += reward
            state = next_state
        
        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso 
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

        # PARTE 2: atualiza Q (e a política, implicitamente)
        Gt = 0
        W = 1
        for (s, a, r) in reversed(ep_trajectory):
            Gt = r + gamma*Gt
            C[s,a] = C[s,a] + W
            delta = W * (Gt - Q[s,a])
            Q[s,a] = Q[s,a] + (1/C[s,a])* delta
            best = choose_action(Q,s)
            if best == choose_actionB(num_actions): break
            W = W*(1/(1/num_actions))

    return sum_rewards_per_ep, Q


### On-Policy

In [54]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores
# da tabela Q, usando uma estratégia epsilon-greedy.
def pi_policy(Q, state, num_actions, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, num_actions)
    else:
        return np.argmax(Q[state])


# Algoritmo Monte-Carlo de Controle, variante "toda-visita".
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_montecarloOnP(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)
    
    num_actions = env.action_space.n
    
    # inicializa a tabela Q toda com zero,
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-discontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        ep_trajectory = []
        
        state = env.reset()
    
        # PARTE 1: executa um episódio completo
        while done != True:   
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios 
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()
                
            # escolhe a próxima ação -- usa epsilon-greedy
            action = pi_policy(Q, state, num_actions, epsilon)
        
            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)
            
            # adiciona a tripla que representa este passo
            ep_trajectory.append( (state, action, reward) )
            
            sum_rewards += reward
            state = next_state
        
        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso 
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

        # PARTE 2: atualiza Q (e a política, implicitamente)
        Gt = 0
        for (s, a, r) in reversed(ep_trajectory):
            Gt = r + gamma*Gt
            delta = Gt - Q[s,a]
            Q[s,a] = Q[s,a] + lr * delta

    return sum_rewards_per_ep, Q

### Execução Off-Policy

In [42]:
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    LR = 0.01
    GAMMA = 0.830525147061507
    EPSILON = 0.05919712699520377

    
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    rewards, Qtable = run_montecarloOffP(env, EPISODES, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarloOffP-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    # plot_result(rewards, r_max_plot, None)

    # test_greedy_Q_policy(env, Qtable, 10, True)
    env.close()

Episode 100 Average Reward (last 100): -200.000
Episode 200 Average Reward (last 100): -235.820
Episode 300 Average Reward (last 100): -235.550
Episode 400 Average Reward (last 100): -504.020
Episode 500 Average Reward (last 100): -539.570
Episode 600 Average Reward (last 100): -594.830
Episode 700 Average Reward (last 100): -756.560
Episode 800 Average Reward (last 100): -936.470
Episode 900 Average Reward (last 100): -1169.300
Episode 1000 Average Reward (last 100): -1097.570
Episode 1100 Average Reward (last 100): -971.660
Episode 1200 Average Reward (last 100): -1042.580
Episode 1300 Average Reward (last 100): -738.920
Episode 1400 Average Reward (last 100): -738.020
Episode 1500 Average Reward (last 100): -845.750
Episode 1600 Average Reward (last 100): -720.650
Episode 1700 Average Reward (last 100): -773.840
Episode 1800 Average Reward (last 100): -666.470
Episode 1900 Average Reward (last 100): -451.730
Episode 2000 Average Reward (last 100): -559.550
Episode 2100 Average Rewar

### Execução On-Policy

#### Otimiza Parâmetros

In [57]:
from numpy.random.mtrand import gamma

import optuna

#from wrappers import DiscreteObservationWrapper


ENV = gym.make("Taxi-v3")


# Esta função faz um treinamento com o Expected-SARSA, usando parâmetros sugeridos pelo Optuna.
# Retorna a média dos retornos dos últimos 100 episódios.
def train_values(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    lr = trial.suggest_uniform('learning_rate', 0.001, 1.0)
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    #bins1 = trial.suggest_int('bins1', 5, 100)
    #bins2 = trial.suggest_int('bins2', 5, 100)
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}")

    # roda o algoritmo e recebe os retornos não-descontados
    #env_wrapper = DiscreteObservationWrapper(ENV, [bins1,bins2])
    (returns, _) = run_montecarloOnP(ENV, 20000, lr, gamma, eps, render=False)
    return sum(returns[-100:])/100 

In [58]:
study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_onpolice', 
                            load_if_exists=True)
study.optimize(train_values, n_trials=20) 

[I 2022-10-01 05:40:30,145] Using an existing study with name 'new_MC_onpolice' instead of creating a new one.
C:\Users\Matheus\AppData\Local\Temp\ipykernel_8052\3070548133.py:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  lr = trial.suggest_uniform('learning_rate', 0.001, 1.0)
C:\Users\Matheus\AppData\Local\Temp\ipykernel_8052\3070548133.py:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
C:\Users\Matheus\AppData\Local\Temp\ipykernel_8052\3070548133.py:18: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/rel


TRIAL #1: eps=0.0944534808641455, gamma=0.14260763782298835
Episode 100 Average Reward (last 100): -486.920
Episode 200 Average Reward (last 100): -570.570
Episode 300 Average Reward (last 100): -444.350
Episode 400 Average Reward (last 100): -358.760
Episode 500 Average Reward (last 100): -301.610
Episode 600 Average Reward (last 100): -282.390
Episode 700 Average Reward (last 100): -265.430
Episode 800 Average Reward (last 100): -268.300
Episode 900 Average Reward (last 100): -251.750
Episode 1000 Average Reward (last 100): -271.670
Episode 1100 Average Reward (last 100): -257.330
Episode 1200 Average Reward (last 100): -270.470
Episode 1300 Average Reward (last 100): -258.230
Episode 1400 Average Reward (last 100): -254.540
Episode 1500 Average Reward (last 100): -279.830
Episode 1600 Average Reward (last 100): -273.890
Episode 1700 Average Reward (last 100): -257.640
Episode 1800 Average Reward (last 100): -251.610
Episode 1900 Average Reward (last 100): -256.200
Episode 2000 Aver

[I 2022-10-01 05:41:01,593] Trial 1 finished with value: -255.65 and parameters: {'learning_rate': 0.900710732616076, 'gamma': 0.14260763782298835, 'epsilon': 0.0944534808641455}. Best is trial 1 with value: -255.65.


Episode 20000 Average Reward (last 100): -255.650

TRIAL #2: eps=0.12202823895690514, gamma=0.9746717628447142
Episode 100 Average Reward (last 100): -495.200
Episode 200 Average Reward (last 100): -763.580
Episode 300 Average Reward (last 100): -721.100
Episode 400 Average Reward (last 100): -728.390
Episode 500 Average Reward (last 100): -705.390
Episode 600 Average Reward (last 100): -561.890
Episode 700 Average Reward (last 100): -647.030
Episode 800 Average Reward (last 100): -626.960
Episode 900 Average Reward (last 100): -673.850
Episode 1000 Average Reward (last 100): -600.680
Episode 1100 Average Reward (last 100): -771.320
Episode 1200 Average Reward (last 100): -668.270
Episode 1300 Average Reward (last 100): -604.910
Episode 1400 Average Reward (last 100): -633.620
Episode 1500 Average Reward (last 100): -649.370
Episode 1600 Average Reward (last 100): -550.670
Episode 1700 Average Reward (last 100): -540.200
Episode 1800 Average Reward (last 100): -567.130
Episode 1900 Ave

[I 2022-10-01 05:41:30,422] Trial 2 finished with value: -567.68 and parameters: {'learning_rate': 0.8190269403881686, 'gamma': 0.9746717628447142, 'epsilon': 0.12202823895690514}. Best is trial 1 with value: -255.65.


...

[I 2022-10-01 05:48:02,320] Trial 18 finished with value: -190.34 and parameters: {'learning_rate': 0.16568977589402378, 'gamma': 0.6389731969911425, 'epsilon': 0.17483556367299188}. Best is trial 16 with value: -2.2.


Episode 20000 Average Reward (last 100): -190.340

TRIAL #19: eps=0.0859457757172045, gamma=0.8997568520596844
Episode 100 Average Reward (last 100): -453.620
Episode 200 Average Reward (last 100): -870.580
Episode 300 Average Reward (last 100): -950.240
Episode 400 Average Reward (last 100): -848.720
Episode 500 Average Reward (last 100): -625.430
Episode 600 Average Reward (last 100): -364.560
Episode 700 Average Reward (last 100): -390.800
Episode 800 Average Reward (last 100): -408.260
Episode 900 Average Reward (last 100): -352.090
Episode 1000 Average Reward (last 100): -308.900
Episode 1100 Average Reward (last 100): -284.150
Episode 1200 Average Reward (last 100): -275.330
Episode 1300 Average Reward (last 100): -262.280
Episode 1400 Average Reward (last 100): -266.240
Episode 1500 Average Reward (last 100): -250.220
Episode 1600 Average Reward (last 100): -251.480
Episode 1700 Average Reward (last 100): -251.120
Episode 1800 Average Reward (last 100): -257.240
Episode 1900 Ave

[I 2022-10-01 05:48:26,090] Trial 19 finished with value: -108.83 and parameters: {'learning_rate': 0.02909295418926345, 'gamma': 0.8997568520596844, 'epsilon': 0.0859457757172045}. Best is trial 16 with value: -2.2.



TRIAL #20: eps=0.12834449095225414, gamma=0.9996571995152939
Episode 100 Average Reward (last 100): -490.070
Episode 200 Average Reward (last 100): -1149.320
Episode 300 Average Reward (last 100): -939.210
Episode 400 Average Reward (last 100): -714.800
Episode 500 Average Reward (last 100): -743.690
Episode 600 Average Reward (last 100): -804.980
Episode 700 Average Reward (last 100): -869.350
Episode 800 Average Reward (last 100): -786.330
Episode 900 Average Reward (last 100): -773.360
Episode 1000 Average Reward (last 100): -847.120
Episode 1100 Average Reward (last 100): -680.320
Episode 1200 Average Reward (last 100): -690.720
Episode 1300 Average Reward (last 100): -736.000
Episode 1400 Average Reward (last 100): -736.290
Episode 1500 Average Reward (last 100): -719.450
Episode 1600 Average Reward (last 100): -805.750
Episode 1700 Average Reward (last 100): -601.190
Episode 1800 Average Reward (last 100): -651.910
Episode 1900 Average Reward (last 100): -683.290
Episode 2000 Av

[I 2022-10-01 05:48:49,600] Trial 20 finished with value: -620.31 and parameters: {'learning_rate': 0.3985605525630849, 'gamma': 0.9996571995152939, 'epsilon': 0.12834449095225414}. Best is trial 16 with value: -2.2.


#### Execução

In [61]:
ENV_NAME = "Taxi-v3"  
#ENV_NAME = "MountainCarContinuous-v0"  
#ENV_NAME = "LunarLander-v2"  
env = gym.make(ENV_NAME)

#parameters: {'learning_rate': 0.007617314337157292, 'gamma': 0.8762195844017038, 'epsilon': 0.1594319424288697}. Best is trial 16 with value: -2.2.
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    LR = 0.007617314337157292
    GAMMA = 0.8762195844017038
    EPSILON = 0.1594319424288697

    
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    rewards, Qtable = run_montecarloOnP(env, EPISODES, LR, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarloOnP-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    # plot_result(rewards, r_max_plot, None)

    # test_greedy_Q_policy(env, Qtable, 10, True)
    env.close()

Episode 100 Average Reward (last 100): -549.290
Episode 200 Average Reward (last 100): -922.890
Episode 300 Average Reward (last 100): -931.770
Episode 400 Average Reward (last 100): -736.940
Episode 500 Average Reward (last 100): -629.520
Episode 600 Average Reward (last 100): -471.410
Episode 700 Average Reward (last 100): -440.600
Episode 800 Average Reward (last 100): -379.740
Episode 900 Average Reward (last 100): -354.050
Episode 1000 Average Reward (last 100): -307.790
Episode 1100 Average Reward (last 100): -316.920
Episode 1200 Average Reward (last 100): -268.470
Episode 1300 Average Reward (last 100): -230.530
Episode 1400 Average Reward (last 100): -222.450
Episode 1500 Average Reward (last 100): -194.800
Episode 1600 Average Reward (last 100): -196.820
Episode 1700 Average Reward (last 100): -177.170
Episode 1800 Average Reward (last 100): -179.720
Episode 1900 Average Reward (last 100): -145.470
Episode 2000 Average Reward (last 100): -124.170
Episode 2100 Average Reward (

### Contínuo

In [44]:
#ENV_NAME = "Taxi-v3"  
ENV_NAME = "MountainCarContinuous-v0"  
#ENV_NAME = "LunarLander-v2"  
env = gym.make(ENV_NAME)

In [45]:
from numpy.random.mtrand import gamma

import optuna

#from wrappers import DiscreteObservationWrapper


ENV = gym.make("Taxi-v3")


# Esta função faz um treinamento com o Expected-SARSA, usando parâmetros sugeridos pelo Optuna.
# Retorna a média dos retornos dos últimos 100 episódios.
def train_values(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    #bins1 = trial.suggest_int('bins1', 5, 100)
    #bins2 = trial.suggest_int('bins2', 5, 100)
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}")

    # roda o algoritmo e recebe os retornos não-descontados
    #env_wrapper = DiscreteObservationWrapper(ENV, [bins1,bins2])
    (returns, _) = run_montecarloOP(env, 20000, gamma, eps, render=False)
    return sum(returns[-100:])/100

In [ ]:
study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_offpolice_cont', 
                            load_if_exists=True)
study.optimize(train_values, n_trials=20) 

In [ ]:
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    LR = 0.01
    GAMMA = 0.95
    EPSILON = 0.1

    
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    rewards, Qtable = run_montecarloOffP(env, EPISODES, LR, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarlo2-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot, None)

    test_greedy_Q_policy(env, Qtable, 10, True)
    env.close()

### Exibe e avalia o agente

In [18]:
record_video(ENV_NAME, model, video_length=1000, prefix='monte-car-off-police')
show_videos('videos', prefix='monte-car-off-police')

NameError: name 'model' is not defined

In [ ]:
#mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=30)

#print(f"Retorno médio: {mean_reward:.2f} +/- {std_reward:.2f}")